In [6]:
def remove_left_recursion(gram, non_terminal):
    alpha = []  
    beta = []  
    for production in gram[non_terminal]: # A → Aα | β
        if production.startswith(non_terminal):
            alpha.append(production[len(non_terminal):])
        else:
            beta.append(production)
    if alpha:
        nonterminal_ = non_terminal + "'"
        gram[non_terminal] = [b + nonterminal_ for b in beta] # β A' for each β
        gram[nonterminal_] = [a + nonterminal_ for a in alpha] + ['ε'] # α A' for each α | ε
    return gram


def subandcheck_for_indirect(gram):
    non_terminals = list(gram.keys())
    for i in range(len(non_terminals)): #  # A → Aα | β OR A → Bα and B → Aβ
        A = non_terminals[i]
        for j in range(i):
            B = non_terminals[j]
            new_productions = []
            for production in gram[A]:
                if production.startswith(B):
                    for prod in gram[B]:
                        new_productions.append(prod + production[len(B):])
                else:
                    new_productions.append(production)
            gram[A] = new_productions   
        gram = remove_left_recursion(gram, A)
    
    return gram

def parse_grammar(input_text):
    grammar = {}
    for line in input_text.splitlines():
        if '→' in line:
            non_terminal, productions = line.split('→')
            non_terminal = non_terminal.strip()
            production_list = [prod.strip() for prod in productions.split('|')]
            grammar[non_terminal] = production_list
    return grammar

"""{
    'E': ['E + T', 'T'],
    'T': ['T * F', 'F'],
    'F': ['(E)', 'id']
}"""


def print_grammar(gram, original_order):
    for non_terminal in original_order:
        print(f"{non_terminal} → {' | '.join(gram[non_terminal])}")
        nonterminal_ = non_terminal + "'"
        if nonterminal_ in gram:
            print(f"{nonterminal_} → {' | '.join(gram[nonterminal_])}")

input_text = """E → E + T | T
T → T * F | F
F → (E) | id"""

input_text2 = """A → A a | A b | c | d
B → B c | d
C → c | d
"""

grammar = parse_grammar(input_text)
original_order = list(grammar.keys()) 
print("Original:")
print_grammar(grammar, original_order)
updated_grammar = subandcheck_for_indirect(grammar)
print("\nPost-removal:")
print_grammar(updated_grammar, original_order)

grammar2 = parse_grammar(input_text2)
original_order2 = list(grammar2.keys()) 
print("\nOriginal:")
print_grammar(grammar2, original_order2)
updated_grammar2 = subandcheck_for_indirect(grammar2)
print("\nPost-removal:")
print_grammar(updated_grammar2, original_order2)

Original:
E → E + T | T
T → T * F | F
F → (E) | id

Post-removal:
E → TE'
E' →  + TE' | ε
T → FT'
T' →  * FT' | ε
F → (E) | id

Original:
A → A a | A b | c | d
B → B c | d
C → c | d

Post-removal:
A → cA' | dA'
A' →  aA' |  bA' | ε
B → dB'
B' →  cB' | ε
C → c | d
